In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# headers to get past captcha 
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}
with requests.Session() as s:
   city = 'dallas/' 
   url = 'https://www.zillow.com/homes/for_sale/'+ city    
   r = s.get(url, headers=req_headers)

In [3]:
# Scrape pages (normally done with splinter but zillow blocks all bots)
i = 1
pages = {}

while i < 20:
    with requests.Session() as s:
        city = 'dallas/'
        url = f'https://www.zillow.com/homes/for_sale/ + {city} + {i} + _p'
        r = s.get(url, headers=req_headers)
        
        # save each page as a variable
        key = i
        soup = bs(r.content, 'html.parser')
        pages[key] = soup
    
        # next page 
        i += 1

In [15]:
address_list = []
price_list = []
list_card = []

# loop through saved pages and scrape info
for i in range(1,20):
    soup = pages[i]

    listing = soup.find_all('ul', class_='photo-cards photo-cards_wow photo-cards_short photo-cards_extra-attribution')
    
    for lists in listing:
        address = lists.find_all('address', class_='list-card-addr')
        price = lists.find_all('div', class_='list-card-price')
        bd_ba_size = lists.find_all('ul', class_='list-card-details')

    for bd in bd_ba_size:
        info = bd.find_all('li', class_='')
        for i in info:
            list_card.append(i.text)

    for a in address:
        address_list.append(a.text)

    for p in price:
        price_list.append(p.text)

In [16]:
# slice list card to extract beds, baths, size
bds = list_card[0::3]
baths = list_card[1::3]
size = list_card[2::3]

In [17]:
# add to df 
house_info = pd.DataFrame({'address': address_list, 
                           'price': price_list,
                           'beds': bds,
                           'baths': baths,
                           'size': size})
house_info

,address,price,beds,baths,size
0,"6220 Bentwood Trl APT 206, Dallas, TX 75252","$175,000",1 bd,2 ba,807 sqft
1,"2827 Curvilinear Ct, Dallas, TX 75227","$245,000",4 bds,2 ba,"2,200 sqft"
2,"132 W Woodin Blvd, Dallas, TX 75224","$275,000",4 bds,2 ba,"1,800 sqft"
3,"7311 Hardwood Trl, Dallas, TX 75249","$209,900",3 bds,2 ba,"1,902 sqft"
4,"3609 Patience Blvd, Dallas, TX 75236","$324,990",4 bds,2 ba,"2,067 sqft"
...,...,...,...,...,...
755,"5505 Ellsworth Ave, Dallas, TX 75206","$729,000",4 bds,5 ba,"2,811 sqft"
756,"10619 Lakemere Dr, Dallas, TX 75238","$499,900",3 bds,2 ba,"1,647 sqft"
757,"818 Cordova St, Dallas, TX 75223","$650,000",3 bds,2 ba,"1,814 sqft"
758,"11727 Welch Rd, Dallas, TX 75229","$699,000",3 bds,3 ba,"2,491 sqft"


## Clean data

In [18]:
# split address
address = house_info['address'].str.split(",", n = 2, expand = True)
  
# separate columns
house_info['address']= address[0]
house_info['city'] = address[1]
house_info['state'] = address[2]

# split zip code 
zipcode = house_info['state'].str.split(n=2, expand = True)
house_info['state'] = zipcode[0]
house_info['zipcode'] = zipcode[1]


In [19]:
# strip 'bds', 'ba', 'sqft'
beds = house_info['beds'].str.split(' ', n = 2, expand = True)
bath = house_info['baths'].str.split(' ',n = 2, expand = True)
size = house_info['size'].str.split(' ', n = 2, expand = True)

house_info['beds'] = beds[0]
house_info['baths'] = bath[0]
house_info['size'] = size[0]


In [20]:
# Convert numers to only numerical and strip price tag
house_info['price'] = house_info['price'].str.replace('$', '').str.replace(',', '')
house_info['size'] = house_info['size'].str.replace(',', '')

house_info.head()

,address,price,beds,baths,size,city,state,zipcode
0,6220 Bentwood Trl APT 206,175000,1,2,807,Dallas,TX,75252
1,2827 Curvilinear Ct,245000,4,2,2200,Dallas,TX,75227
2,132 W Woodin Blvd,275000,4,2,1800,Dallas,TX,75224
3,7311 Hardwood Trl,209900,3,2,1902,Dallas,TX,75249
4,3609 Patience Blvd,324990,4,2,2067,Dallas,TX,75236
